In [366]:
# Import pandas library 
import pandas as pd 
import numpy as np
from ast import literal_eval

data = [[12346, '(135:2345678, 212:4354670, 198:9876545)', '(Flag1, Flag2, Flag3)'],
[12345, '(136:2343678, 212:4354670, 198:9876541, 199:9876535)', '(Flag1, Flag4)']]

df = pd.DataFrame(data,columns=['id','relationships','flags'])
df = df.set_index('id')
df

,relationships,flags
id,,
12346,"(135:2345678, 212:4354670, 198:9876545)","(Flag1, Flag2, Flag3)"
12345,"(136:2343678, 212:4354670, 198:9876541, 199:98...","(Flag1, Flag4)"


In order to summarize or count by columns, we need to improve our data structure, in some way that we can apply group by operations with department, relationships or flags.

We will convert our relationships and flags columns from string type to a python list of strings. So, the flags column will be a python list of flags, and the relationships column will be a python list of relations.

In [367]:
df['relationships'] = df['relationships'].str.replace('\(','').str.replace('\)','')
df['relationships'] = df['relationships'].str.split(',')

df['flags'] = df['flags'].str.replace('\(','').str.replace('\)','')
df['flags'] = df['flags'].str.split(',')
df

,relationships,flags
id,,
12346,"[135:2345678, 212:4354670, 198:9876545]","[Flag1, Flag2, Flag3]"
12345,"[136:2343678, 212:4354670, 198:9876541, 199...","[Flag1, Flag4]"


With our `relationships` column converted to list, we can create a new dataframe as much columns
as relations in that lists we have.

In [368]:
rel = pd.DataFrame(df['relationships'].values.tolist(), index=rel.index)
rel

,0,1,2,3
id,,,,
12346,135:2345678,212:4354670,198:9876545,None
12345,136:2343678,212:4354670,198:9876541,199:9876535


After that we need to stack our  columns preserving its index, so we will use pandas 
multi_index: the `id` and the relation column number(0,1,2,3)

In [369]:
relations = rel.stack()
relations.index.names = ['id','relation_number']
relations

id     relation_number
12346  0                   135:2345678
       1                   212:4354670
       2                   198:9876545
12345  0                   136:2343678
       1                   212:4354670
       2                   198:9876541
       3                   199:9876535
dtype: object

At this moment we have all of our relations in rows, but still we can't group by using 
 `relation_type` feature. So we will split our relations data in two columns: `relation_type` and `department` using `:`.

In [370]:
clear_relations = relations.str.split(':')
clear_relations = pd.DataFrame(clear_relations.values.tolist(), index=clear_relations.index,columns=['relation_type','department'])
clear_relations

relation_type department
id    relation_number                         
12346 0                         135    2345678
      1                         212    4354670
      2                         198    9876545
12345 0                         136    2343678
      1                         212    4354670
      2                         198    9876541
      3                         199    9876535

Our relations are ready to analyze, but our flags structure still is very useless. So we will convert the flag list, to columns and after that we will stack them.

In [377]:
flags = pd.DataFrame(df['flags'].values.tolist(), index=rel.index)
flags = flags.stack()
flags.index.names = ['id','flag_number']
flags

0
id    flag_number        
12346 0             Flag1
      1             Flag2
      2             Flag3
12345 0             Flag1
      1             Flag4

Voilá!, It's all ready to analyze!.

So, how many relations from each type we have, and wich one is the biggest.

In [371]:
print('all unique relations:' + str(len(clear_relations['relation_type'].unique())))
clear_relations.groupby('relation_type').agg('count')['department'].sort_values(ascending=False)

all unique relations:5


relation_type
 212    2
 198    2
136     1
135     1
 199    1
Name: department, dtype: int64

In [372]:
print('all unique departments:' + str(len(clear_relations['department'].unique())))
clear_relations.groupby('department').agg('count')['relation_type'].sort_values(ascending=False)

all unique departments:6


department
4354670    2
9876545    1
9876541    1
9876535    1
2345678    1
2343678    1
Name: relation_type, dtype: int64